# Filter Analysis

The following notebook allows you to gain familiarity with multi-stage filters.

The slider bars controll the cutoff frequencies for the highpass filter (HPF) and the lowpass filter (LPF).

There is also a slider bar for the filter order (N). This number indicates how many identical HPF and LPF stages we want to use in a row to get a sharper filter response.
For this example, the total filter order is 2*N:  there are N identical lowpass filters, followed by N identical highpass filters, all separated by buffers to prevent loading effects.


In the time-domain plots:
 - red is the 60Hz noise
 - green is the 600Hz desired signal
 - blue is the 60kHz noise -- Note: it is plotted very faintly, if not you cannot see the other signals!
 - black is the sum of all the voltages
 
 
 If there were no noise present, the black signal (total voltage) would overlap the green signal (desired signal) exactly.
 However, because noise is present the black signal deviates from the green, as can be seen for the input voltage waveform.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib
matplotlib.interactive(True)
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
import scipy
from scipy import signal

In [ ]:
# Definitions and slider limits

# Slider limits
lp_min = 2*np.pi*1
lp_max = 2*np.pi*1e5
hp_min = 2*np.pi*1
hp_max = 2*np.pi*1e5
lp_init = 2*np.pi*600*10
hp_init = 2*np.pi*120

# Frequencies
w_60 = 2*np.pi*60
w_600 = 2*np.pi*600
w_60k = 2*np.pi*60e3

in_60_A = 20e-3
in_600_A = 1e-3
in_60k_A = 10e-3

t = np.arange(0, 2*2*np.pi/w_60, 1/10*2*np.pi/w_60k)
ws = [w_60, w_600, w_60k]


##########
# Helper function
##########
def bode_data(afilter):
    my_wbode, my_magbode, my_phasebode = signal.bode(afilter, np.logspace(1, 6))
    return my_wbode, my_magbode, my_phasebode


##########
# Initial plots
##########

# Initial filter
filter = signal.ZerosPolesGain([0]*1, 
                               [lp_init, hp_init]*1, 
                               -np.power(np.abs(lp_init), 1))


# Bode plots
wbode1, magbode1, phasebode1 = bode_data(filter)

fig1, (ax1, ax2) = plt.subplots(2, 1)
plt.subplots_adjust(top=0.65)

ax1.semilogx(wbode1/(2*np.pi), magbode1)
ax1.set_ylabel('Magnitude [dB]')
ax1.grid()
ax2.semilogx(wbode1/(2*np.pi), phasebode1)
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Phase [deg]')
ax2.grid()

##########
# Sliders
##########
ax_lp = plt.axes([0.2, 0.95, 0.7, 0.03])
ax_hp = plt.axes([0.2, 0.9, 0.7, 0.03])
ax_N = plt.axes([0.2, 0.85, 0.7, 0.03])

w_lp_slider = Slider(ax_lp, 'log(f_lowpass)', 
                     np.log10(lp_min), np.log10(lp_max), 
                     valinit=np.log10(lp_init))
w_hp_slider = Slider(ax_hp, 'log(f_highpass)', 
                     np.log10(hp_min), np.log10(hp_max), 
                     valinit=np.log10(hp_init))
N_slider = Slider(ax_N, 'Filter Order',
                  1, 6, 
                  valinit=1,
                  valstep=1
                 )

# Show slider values
textstr = '\n'.join([
    f'f_lowpass = {np.abs(lp_init)/(2*np.pi)/1000:0.3f} kHz',
    f'f_highpass = {np.abs(hp_init)/(2*np.pi)/1000:0.3f} kHz',
    f'Filter order = {int(1)}'
])
ax1.text(0.0, 1.7, textstr, transform=ax1.transAxes, fontsize=14,
                verticalalignment='top')

####
# Filter response
####

# Input plot
_, Hs = signal.freqresp(filter, ws)
in_60 = in_60_A * np.cos(w_60*t)
in_600 = in_600_A * np.cos(w_600*t)
in_60k = in_60k_A * np.cos(w_60k*t)
in_total = in_60 + in_600 + in_60k

fig3,ax3 = plt.subplots()
ax3.plot(t, in_60, 'r-', label='60Hz noise')
ax3.plot(t, in_600, 'g', label='600Hz desired')
ax3.plot(t, in_60k, 'b--', linewidth=0.01, label='60kHz noise')
ax3.plot(t, in_total, 'k', linewidth=0.05, label='Total signal')
ax3.legend()
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Input Voltages [V]')


# Output plot
out_60 = in_60_A*np.abs(Hs[0])*np.cos(w_60*t + np.angle(Hs[0]))
out_600 = in_600_A*np.abs(Hs[1])*np.cos(w_600*t + np.angle(Hs[1]))
out_60k = in_60k_A*np.abs(Hs[2])*np.cos(w_60k*t + np.angle(Hs[2]))
out_total = out_60 + out_600 + out_60k

fig4, ax4 = plt.subplots()
ax4.plot(t, out_60, 'r-', label='60Hz noise')
ax4.plot(t, out_600, 'g', label='600Hz desired')
ax4.plot(t, out_60k, 'b--', linewidth=0.01, label='60kHz noise')
ax4.plot(t, out_total, 'k', linewidth=2, label='Total signal')
ax4.legend()
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('Output Voltages [V]')









##########
# helper functions and update
##########

    
def update(val):
    # Update values for the filter
    w_lp_val = -1*10**w_lp_slider.val
    w_hp_val = -1*10**w_hp_slider.val
    N_val = int(N_slider.val)
    
    # Redefine the filter
    filter = signal.ZerosPolesGain([0]*N_val, 
                                   [w_lp_val, w_hp_val]*N_val, 
                                   -np.power(np.abs(w_lp_val), N_val))
    
    # New Bode plot and display it
    wbode1, magbode1, phasebode1 = bode_data(filter)
    ax1.clear()
    ax1.semilogx(wbode1/(2*np.pi), magbode1)
    ax1.set_ylabel('Magnitude [dB]')
    ax1.grid()
    ax2.clear()
    ax2.semilogx(wbode1/(2*np.pi), phasebode1)
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Phase [deg]')
    ax2.grid()
    
    # Show slider values
    textstr = '\n'.join([
        f'f_lowpass = {np.abs(w_lp_val)/(2*np.pi)/1000:0.3f} kHz',
        f'f_highpass = {np.abs(w_hp_val)/(2*np.pi)/1000:0.3f} kHz',
        f'Filter order = {int(N_val)}'
    ])
    ax1.text(0.0, 1.7, textstr, transform=ax1.transAxes, fontsize=14,
                verticalalignment='top')
    
    # Recalculate output
    _, Hs = signal.freqresp(filter, ws)
    
    out_60 = in_60_A*np.abs(Hs[0])*np.cos(w_60*t + np.angle(Hs[0]))
    out_600 = in_600_A*np.abs(Hs[1])*np.cos(w_600*t + np.angle(Hs[1]))
    out_60k = in_60k_A*np.abs(Hs[2])*np.cos(w_60k*t + np.angle(Hs[2]))
    out_total = out_60 + out_600 + out_60k

    ax4.clear()
    ax4.plot(t, out_60, 'r-', label='60Hz noise')
    ax4.plot(t, out_600, 'g', label='600Hz desired')
    ax4.plot(t, out_60k, 'b--', linewidth=0.01, label='60kHz noise')
    ax4.plot(t, out_total, 'k', linewidth=2, label='Total signal')
    ax4.legend()
    ax4.set_xlabel('Time [s]')
    ax4.set_ylabel('Output Voltages [V]')
    
w_lp_slider.on_changed(update)
w_hp_slider.on_changed(update)
N_slider.on_changed(update)

plt.show()
